## 0.1.Spark Context

In [1]:
%%configure -f
{
    "conf":  { 
             "spark.jars":"hdfs:///user/hadoop/spark-tfrecord_2.12-0.3.0.jar, hdfs:///user/hadoop/httpclient-4.5.9.jar, hdfs:///user/hadoop/hudi-spark-bundle.jar,hdfs:///user/hadoop/spark-avro.jar",
             "spark.sql.hive.convertMetastoreParquet":"false",     
             "spark.serializer":"org.apache.spark.serializer.KryoSerializer",
             "spark.dynamicAllocation.executorIdleTimeout": 3600,
             "spark.executor.memory": "4G",
             "spark.executor.cores": 2,
             "spark.dynamicAllocation.initialExecutors":2,
             "spark.sql.hive.convertMetastoreParquet": "false"
           } 
}

 ## 0.2 Hudi configuration

In [2]:
# General Constants
HUDI_FORMAT = "org.apache.hudi"
TABLE_NAME = "hoodie.table.name"
RECORDKEY_FIELD_OPT_KEY = "hoodie.datasource.write.recordkey.field"
PRECOMBINE_FIELD_OPT_KEY = "hoodie.datasource.write.precombine.field"
OPERATION_OPT_KEY = "hoodie.datasource.write.operation"
BULK_INSERT_OPERATION_OPT_VAL = "bulk_insert"
UPSERT_OPERATION_OPT_VAL = "upsert"
BULK_INSERT_PARALLELISM = "hoodie.bulkinsert.shuffle.parallelism"
UPSERT_PARALLELISM = "hoodie.upsert.shuffle.parallelism"
S3_CONSISTENCY_CHECK = "hoodie.consistency.check.enabled"
HUDI_CLEANER_POLICY = "hoodie.cleaner.policy"
KEEP_LATEST_COMMITS = "KEEP_LATEST_COMMITS"
HUDI_COMMITS_RETAINED = "hoodie.cleaner.commits.retained"
PAYLOAD_CLASS_OPT_KEY = "hoodie.datasource.write.payload.class"
EMPTY_PAYLOAD_CLASS_OPT_VAL = "org.apache.hudi.EmptyHoodieRecordPayload"

# Hive Constants
HIVE_SYNC_ENABLED_OPT_KEY="hoodie.datasource.hive_sync.enable"
HIVE_PARTITION_FIELDS_OPT_KEY="hoodie.datasource.hive_sync.partition_fields"
HIVE_ASSUME_DATE_PARTITION_OPT_KEY="hoodie.datasource.hive_sync.assume_date_partitioning"
HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY="hoodie.datasource.hive_sync.partition_extractor_class"
HIVE_TABLE_OPT_KEY="hoodie.datasource.hive_sync.table"
HIVE_DATABASE_OPT_KEY="hoodie.datasource.hive_sync.database"

# Partition Constants
NONPARTITION_EXTRACTOR_CLASS_OPT_VAL="org.apache.hudi.hive.NonPartitionedExtractor"
MULTIPART_KEYS_EXTRACTOR_CLASS_OPT_VAL="org.apache.hudi.hive.MultiPartKeysValueExtractor"
KEYGENERATOR_CLASS_OPT_KEY="hoodie.datasource.write.keygenerator.class"
NONPARTITIONED_KEYGENERATOR_CLASS_OPT_VAL="org.apache.hudi.NonpartitionedKeyGenerator"
COMPLEX_KEYGENERATOR_CLASS_OPT_VAL="org.apache.hudi.ComplexKeyGenerator"
PARTITIONPATH_FIELD_OPT_KEY="hoodie.datasource.write.partitionpath.field"
MERGE_SCHEME="hoodie.mergeSchema"

#Incremental Constants
VIEW_TYPE_OPT_KEY="hoodie.datasource.view.type"
BEGIN_INSTANTTIME_OPT_KEY="hoodie.datasource.read.begin.instanttime"
VIEW_TYPE_INCREMENTAL_OPT_VAL="incremental"
END_INSTANTTIME_OPT_KEY="hoodie.datasource.read.end.instanttime"

hudi_options={}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
59,application_1639355815254_1097,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
#spark.sql("drop table emr_feature_store.customer_base")
#spark.sql("drop table emr_feature_store.customer_advance")
#spark.sql("drop database emr_feature_store")
#spark.sql("show create table emr_feature_store.customer_base as serde").show(10000,False)
#spark.sql("select customer_id,city_code,state_code,country_code,dt  from emr_feature_store.customer_base ").show(10000,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+---------+----------+------------+--------+
|customer_id|city_code|state_code|country_code|dt      |
+-----------+---------+----------+------------+--------+
|109367     |2        |5         |3           |20210211|
|573291     |1        |49        |2           |20210211|
|828412     |4        |31        |3           |20210307|
|828415     |3        |5         |2           |20210307|
|828413     |3        |31        |1           |20210307|
|828414     |3        |31        |5           |20210307|
|109382     |2        |40        |2           |20210212|
|109366     |2        |40        |4           |20210312|
|109365     |2        |5         |5           |20210311|
|124013     |4        |5         |2           |20210305|
|828400     |3        |31        |2           |20210304|
|124014     |4        |31        |1           |20210306|
+-----------+---------+----------+------------+--------+

## 1 create emr feature store

In [6]:
import os
from pyspark.sql import SparkSession
##########feature store 相关#########
feature_store_name='emr_feature_store'
feature_store_s3_path='s3://emrfssampledata/emr_feature_store/'
#####################################
sc = SparkSession.builder\
            .enableHiveSupport()\
            .getOrCreate()
sql="create database if not exists "+feature_store_name+" location '"+feature_store_s3_path+"'"
sc.sql(sql)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

## 2 create emr feature group
note： in emr 6.5, we can use spark sql to directly create hudi table ("using hudi"), no need to create and sychronize hive metadata

In [7]:
##########feature store 相关#########
spark.sql("use "+feature_store_name+";")
feature_group_name = "customer_base"
feature_unique_key = 'customer_id'
feature_partition_key = 'dt'
feature_group_path = feature_store_s3_path+feature_group_name
features="customer_id int, city_code   int, state_code  string ,country_code string, dt string"

#feature_group_name = "customer_advance"
#feature_unique_key = 'customer_id'
#feature_partition_key = 'dt'
#feature_group_path = feature_store_s3_path+feature_group_name
#features="customer_id int",age int,diabetes string, ejection_fraction string,high_blood_pressure string,platelets string,sex string, smoking string,death_event string,dt  string
#####################################
sql="""CREATE external TABLE if not exists {0}( 
{4})  
PARTITIONED BY ( 
  dt) 
ROW FORMAT SERDE 
  'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe' 
STORED AS INPUTFORMAT 
  'org.apache.hudi.hadoop.HoodieParquetInputFormat' 
OUTPUTFORMAT 
  'org.apache.hadoop.hive.ql.io.parquet.MapredParquetOutputFormat' 
TBLPROPERTIES ('feature_unique_key'='{1}','feature_partition_key'='{2}')
LOCATION '{3}'"""
sql=sql.format(feature_group_name,feature_unique_key,feature_partition_key,feature_group_path,features)
print("sql=="+sql)
spark.sql(sql)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

sql==CREATE external TABLE if not exists customer_base( 
customer_id int, city_code   int, state_code  string ,country_code string, dt string)  
PARTITIONED BY ( 
  dt) 
ROW FORMAT SERDE 
  'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe' 
STORED AS INPUTFORMAT 
  'org.apache.hudi.hadoop.HoodieParquetInputFormat' 
OUTPUTFORMAT 
  'org.apache.hadoop.hive.ql.io.parquet.MapredParquetOutputFormat' 
TBLPROPERTIES ('feature_unique_key'='customer_id','feature_partition_key'='dt')
LOCATION 's3://emrfssampledata/emr_feature_store/customer_base'
DataFrame[]

## 3 ingestion data into feature group 
note :provide feature partition key and feature primary key
#feature_unique_key,特征唯一标识列，比如：uuid 
#feature_partition_key, 特征group表分区字段，比如dt
#hoodie.datasource.write.operation, 操作特征group表方式，设置为upsert支持update
#hoodie.mergeSchema,设置为"true" 支持新增feature field
#只有第一次创建表才使用overwrite，其他情况都用append

In [8]:
# 定义源数据路径。
dataGen = "s3://emrfssampledata/feature_store_customer_base.csv"
#dataGen = "s3://emrfssampledata/feature_store_customer_advance.csv"
df=sc.read.format("csv").option("header", "true").load(dataGen)

##########feature store 相关#########
feature_group_name = 'customer_base'
feature_unique_key = 'customer_id'
feature_partition_key='dt'
basePath = feature_store_s3_path+"/"+feature_group_name
#####################################

# hudi操作配置预置参数，
hudi_options = {
   TABLE_NAME: feature_group_name,
   RECORDKEY_FIELD_OPT_KEY: feature_unique_key,
   PARTITIONPATH_FIELD_OPT_KEY: feature_partition_key,
   OPERATION_OPT_KEY: UPSERT_OPERATION_OPT_VAL,
   PRECOMBINE_FIELD_OPT_KEY: feature_partition_key,
   MERGE_SCHEME: "true",
   HIVE_TABLE_OPT_KEY:feature_group_name,
   HIVE_SYNC_ENABLED_OPT_KEY:"true",
   HIVE_PARTITION_FIELDS_OPT_KEY:feature_partition_key,
   HIVE_DATABASE_OPT_KEY:feature_store_name
  #'hoodie.datasource.write.table.name': feature_group_name,  
  #HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY:NONPARTITION_EXTRACTOR_CLASS_OPT_VAL,
  #KEYGENERATOR_CLASS_OPT_KEY:NONPARTITIONED_KEYGENERATOR_CLASS_OPT_VAL
}

print(hudi_options)
df=df.withColumn("customer_id",df["customer_id"].cast("int"))
df=df.withColumn("city_code",df["city_code"].cast("int"))
df.write.format("org.apache.hudi"). \
  options(**hudi_options). \
  mode("overwrite"). \
  save(basePath)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'hoodie.table.name': 'customer_base', 'hoodie.datasource.write.recordkey.field': 'customer_id', 'hoodie.datasource.write.partitionpath.field': 'dt', 'hoodie.datasource.write.operation': 'upsert', 'hoodie.datasource.write.precombine.field': 'dt', 'hoodie.mergeSchema': 'true', 'hoodie.datasource.hive_sync.table': 'customer_base', 'hoodie.datasource.hive_sync.enable': 'true', 'hoodie.datasource.hive_sync.partition_fields': 'dt', 'hoodie.datasource.hive_sync.database': 'emr_feature_store'}

## 4 add new feature in feature group 
#标准sql add column查询操作
#hudi schemaMerge参数默认打开，写入自动会合并分区parquet中的schema

In [11]:
################### add feature column##########################

##########feature store 相关#########
new_feature="identify_code string"
sc.sql("alter table emr_feature_store.customer_base add COLUMNS  ("+new_feature+")")
#####################################

dataGen = "s3://emrfssampledata/feature_store_customer_base_update.csv"
df_updates=sc.read.format("csv").option("header", "true").load(dataGen)
df_updates=df_updates.withColumn("customer_id",df_updates["customer_id"].cast("int"))
df_updates=df_updates.withColumn("city_code",df_updates["city_code"].cast("int"))
df_updates.write.format("org.apache.hudi"). \
  options(**hudi_options). \
  mode("Append"). \
  save(basePath)
sc.sql("select customer_id,city_code,state_code,country_code,dt,identify_code from emr_feature_store.customer_base").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+---------+----------+------------+--------+-------------+
|customer_id|city_code|state_code|country_code|      dt|identify_code|
+-----------+---------+----------+------------+--------+-------------+
|     109367|        2|         5|           3|20210211|  51109873794|
|     573291|        1|        49|           2|20210211|  51109873783|
|     828412|        4|        31|           3|20210307|  51109873788|
|     828415|        3|         5|           2|20210307|         null|
|     828413|        3|        31|           1|20210307|  51109873789|
|     828414|        3|        31|           5|20210307|  51109873790|
|     109382|        2|        40|           2|20210212|  51109873784|
|     828400|        3|        31|           2|20210304|         null|
|     124014|        4|        31|           1|20210306|         null|
|     109366|        2|        40|           4|20210312|  51109873793|
|     109365|        2|         5|           5|20210311|         null|
|     

## 5 upsert feature group 
#hudi操作配置预置参数修改为upsert

In [13]:
hudi_options[OPERATION_OPT_KEY]=UPSERT_OPERATION_OPT_VAL
##########feature store 相关#########
updates = [(124014, 100000,"40","4","20210306","51100212121212")]
#####################################

df_updates=spark.createDataFrame(updates, schema=["customer_id","city_code","state_code","country_code","dt","identify_code"])
df_updates=df_updates.withColumn("customer_id",df_updates["customer_id"].cast("int"))
df_updates=df_updates.withColumn("city_code",df_updates["city_code"].cast("int"))
df_updates.write.format("org.apache.hudi"). \
  options(**hudi_options). \
  mode("append"). \
  save(basePath)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 6 time travel & filter & join query feature group 
#增量查询，设置hoodie_commit_time起始时间
#标准sql filter & join查询操作

In [14]:
##############time travel版本查询#####################################
beginTime = '20220112000000'
endTime = '20220122235959'

spark.sql("select customer_id,city_code,state_code,country_code,dt,identify_code from emr_feature_store.customer_base where city_code=2 ").show()
spark.sql("select customer_id,city_code,state_code,country_code,dt from  emr_feature_store.customer_base where _hoodie_commit_time>'"+beginTime\
          +"' and _hoodie_commit_time <'"+endTime+"'").show()
#spark.sql("select a.customer_id,a.city_code,a.country_code,a.dt,a.identify_code,b.age,b.platelets,b.sex from  emr_feature_store.customer_base a left join emr_feature_store.customer_advance b where a.customer_id = b.customer_id and a._hoodie_commit_time>'"+beginTime\
#          +"' and a._hoodie_commit_time <'"+endTime+"'").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+---------+----------+------------+--------+-------------+
|customer_id|city_code|state_code|country_code|      dt|identify_code|
+-----------+---------+----------+------------+--------+-------------+
|     109366|        2|        40|           4|20210312|  51109873793|
|     109382|        2|        40|           2|20210212|  51109873784|
|     109365|        2|         5|           5|20210311|         null|
|     109367|        2|         5|           3|20210211|  51109873794|
+-----------+---------+----------+------------+--------+-------------+

+-----------+---------+----------+------------+--------+
|customer_id|city_code|state_code|country_code|      dt|
+-----------+---------+----------+------------+--------+
|     124014|   100000|        40|           4|20210306|
|     109382|        2|        40|           2|20210212|
|     828412|        4|        31|           3|20210307|
|     828415|        3|         5|           2|20210307|
|     828413|        3|        3

## 7 train dataset retrive 
#支持tfrecord、libsvm，csv
#sql 查询后dataframe操作

In [15]:
#############retrieve 特征group数据做为train dataset######################
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint

df=spark.sql("select customer_id,city_code,state_code,country_code,identify_code,dt from  emr_feature_store.customer_base where dt >'20210301'")
format='tfrecord'
path="s3://emrfssampledata/traindataset/output"
if format == 'tfrecord':
    path=path+".tfrecord"
    df.write.format("tfrecord").option("recordType", "Example").mode("overwrite").save(path)
elif format == 'libsvm':
    path=path+".libsvm"
    convDf = df.rdd.map(lambda line: LabeledPoint(line[0],[line[1:]]))
    MLUtils.saveAsLibSVMFile(convDf, path)
elif format == 'csv':
    path=path+".csv"
    df.write.format("csv").mode("overwrite").save(path)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…